In [ ]:
import speech_recognition as sr

def convert_wav_to_text(wav_file: str):
    output_file = wav_file.replace(".wav", ".txt")
    output_file = output_file.replace("inputs/", "outputs/")
    recognizer = sr.Recognizer()
    
    with sr.AudioFile(wav_file) as source:
        print("Przetwarzanie pliku audio...")
        audio_data = recognizer.record(source)
        
        try:
            text = recognizer.recognize_google(audio_data)
            print("Rozpoznany tekst:")
            print(text)
            
            with open(output_file, "w", encoding="utf-8") as f:
                f.write(text)
                
            print(f"Tekst zapisany do pliku: {output_file}")
        except sr.UnknownValueError:
            print("Nie udało się rozpoznać mowy.")
        except sr.RequestError as e:
            print(f"Błąd żądania do API rozpoznawania mowy: {e}")


In [ ]:
import os
import wave
import json
import speech_recognition as sr
from vosk import Model, KaldiRecognizer
from textblob import TextBlob

def convert_wav_to_text_vosk(wav_file: str):
    output_file = wav_file.replace(".wav", ".txt").replace("inputs/", "outputs/")
    model_path = "model"
    
    if not os.path.exists(model_path):
        print("Model Vosk nie został znaleziony!")
        return
    
    model = Model(model_path)
    recognizer = KaldiRecognizer(model, 16000)
    
    with wave.open(wav_file, "rb") as wf:
        if wf.getnchannels() != 1 or wf.getsampwidth() != 2 or wf.getcomptype() != "NONE":
            print("Nieobsługiwany format pliku audio")
            return
        
        while True:
            data = wf.readframes(4000)
            if len(data) == 0:
                break
            recognizer.AcceptWaveform(data)
    
    result = json.loads(recognizer.Result())
    text = result.get("text", "")
    
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(text)
    
    print(f"Tekst zapisany do pliku: {output_file}")
    return text

def analyze_sentiment(text: str):
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity  # od -1 do 1
    subjectivity = blob.sentiment.subjectivity  # od 0 do 1
    
    sentiment = "Neutralny"
    if polarity > 0:
        sentiment = "Pozytywny"
    elif polarity < 0:
        sentiment = "Negatywny"
    
    print(f"Analiza sentymentu: {sentiment} (Polaryzacja: {polarity}, Subiektywność: {subjectivity})")
    return sentiment, polarity, subjectivity

def process_audio_files():
    input_files = ["inputs/input1.wav", "inputs/input2.wav", "inputs/input3.wav"]
    
    for file in input_files:
        print(f"Przetwarzanie pliku: {file}")
        
        text = convert_wav_to_text(input_wav)
        textVosk = convert_wav_to_text_vosk(file)
        if textVosk:
            analyze_sentiment(textVosk)

if __name__ == "__main__":
    process_audio_files()
